In [150]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [151]:
template = """You are an evaluator who rates the quality of translations from English to German. The English sentence is '{input_text}' and the translated sentence is ‘{output_text}’. Adhere to the following instructions to complete the evaluation:
1. Evaluate each translation in terms of appropriateness, content, grammar and relevance.
2. Evaluate the quality of translation with a score between 0 and 10.
3. Fill in "Evaluation of appropriateness", "Evaluation of content", "Evaluation of grammar", "Evaluation of relevance" and "your score" of output_schema.
Here is the output schema.
output_schema = {{
  "appropriateness": "Evaluation of appropriateness",
  "content": "Evaluation of content",
  "grammar": "Evaluation of grammar",
  "relevance": "Evaluation of relevance",
  "score": "your score"}} """

prompt = PromptTemplate(template=template, input_variables=["input_text", "output_text"])

In [152]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [163]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="./model/llama-2-7b-chat.Q2_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

root ::= object 
object ::= [{] ws object_11 [}] 
value ::= object | array | string | number | boolean | [n] [u] [l] [l] 
array ::= [[] ws array_15 []] 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_20 ws 
boolean ::= boolean_21 ws 
ws ::= ws_23 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= [-] | 
number_20 ::= [0-9] number_20 | [0-9] 
boolean_21 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] 
ws_22 ::= [ <U+0009><U+000A>] ws 
ws_23 ::= ws_22 | 


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./model/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q3_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q3_K     [  4096,  4096,     1,     1 ]
ll

In [164]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
input_text = "I always say when one door closes, another one opens."
output_text = "저는 항상 한 문이 닫히면 다른 문이 열린다고 말합니다."
ans = llm_chain.run({"input_text":input_text, "output_text":output_text})

ggml_metal_free: deallocating


In [165]:
print(ans)

{ 

"appropriateness": 7,
"content": 6,
"grammar": 8,
"relevance": 7,
"Score": 7}


In [166]:
ans = ans.split('\n')
ans = [txt for txt in ans if txt != ""]
print(ans)

In [167]:
app = ans[0].replace('Appropriateness: ', '').replace('/10', '')
app_reason = ans[1]
flu = ans[2].replace('Fluency: ', '').replace('/10', '')
flu_reason = ans[3]

7


In [48]:
print(app, app_reason, flu, flu_reason, sep='\n')

8
The translated sentence "저는 늘 한 문이 닫히면 다른 문이 열린다고 말하곤 합니다." is appropriate for the context of a conversation about life experiences and lessons learned. The Korean translation uses common, everyday language that is easily understood by native speakers. However, the sentence may not be as sophisticated or nuanced as some other translations, which could impact its appropriateness in certain situations.
7
The translated sentence "저는 늘 한 문이 닫히면 다른 문이 열린다고 말하곤 합니다." is reasonably well-translated, but there are a few areas where the fluency could be improved. For example, the phrase "한 문이" (one door) is not as natural in Korean as it would be in English. Additionally, the verb tenses are
